In [17]:
import requests
import json
from pytils.translit import slugify
import requests
from tqdm.notebook import tqdm
import pandas as pd
from json import load, loads, dumps
import re

In [19]:
data_path = "regions_v3.csv"

data = pd.read_csv(data_path).values
waterplaces_map = {}
for i in tqdm(range(len(data))):
    row = {
        'name': data[i][0],
        'stat': data[i][1],
        'sities': data[i][2],
    }
    waterplaces = loads(data[i][3])
    row['stat'] = loads(row['stat'])
    row['stat']['main'] = {fish.capitalize(): row['stat']['main'][fish] for fish in row['stat']['main']}
    row['stat'] = dumps(row['stat'])
    r = requests.post('https://back.fishow.ru/api/region/', json=row)
    slug = r.json()['slug']
    for waterplace in waterplaces:
        key = (waterplace['type'], waterplace['waterplace'])
        if not key in waterplaces_map:
            waterplaces_map[key] = []
        waterplaces_map[key].append(slug)

In [23]:
data_path = "waterplaces_v6.csv"

data = pd.read_csv(data_path).values

for i in tqdm(range(len(data))):
    key = (data[i][1], data[i][0])
    slugs = []
    if key in waterplaces_map:
        slugs = waterplaces_map[key]
    row = {
        'blogs': [],
        'news': [],
        'name': data[i][0],
        'type': data[i][1],
        'stat': data[i][2] if not pd.isnull(data[i][2]) else "{}",
        'regions': slugs,
        'fish': data[i][3]
    }
#     print(slug)
    r = requests.post('https://back.fishow.ru/api/waterplace_nature/', json=row)